In [6]:
import pandas as pd
import numpy as np
import random
import os
import functions_data_transformation as functions

In [7]:
weather_hourly = pd.read_csv(os.getcwd() + '\\' + 'weather_hourly_darksky.csv')
weather_hourly.time = pd.to_datetime(weather_hourly.time)
weather = weather_hourly[['time','temperature','windSpeed','precipType','icon','summary']]

block_numbers_list = functions.get_random_blocks(num_blocks = 1, seed=84)
block_numbers_list

[93]

In [8]:
for num in block_numbers_list:
        df, house_select = functions.get_random_household(block_numer = num , seed = 42)
        df_block = functions.transform_half_in_hourly(df)
        df_block_weather = functions.add_weater_data(df_block,weather)
        df_block_weather = functions.add_bool_weather_missing_values(df_block_weather)
        df_holidays = functions.add_weekend_holidays(df_block_weather)

        blocks, dict_labels = functions.feature_eng_function(df_holidays)
        blocks = blocks.reset_index(drop=True).reset_index()
        blocks = blocks.rename(columns={'index':'time_idx'})
        blocks = blocks.drop(columns='time')

        functions.save_json(dict_labels,house_select)
        blocks.to_csv(f'dataframe_model\{house_select}.csv',index=False)

AttributeError: module 'functions_data_transformation' has no attribute 'add_weekend_holidays'

In [ ]:
info_dataset = pd.read_csv('informations_households.csv')

In [9]:
acorn_dict = {'luxuryLife' : ['ACORN-A', 'ACORN-B', 'ACORN-C'],
                'establishedAffluence' : ['ACORN-D', 'ACORN-E'],
                'thrivingNeighbour' : ['ACORN-F', 'ACORN-G', 'ACORN-H', 'ACORN-I', 'ACORN-J'],
                'steadfastCommunities' : ['ACORN-K','ACORN-L', 'ACORN-M', 'ACORN-N', 'ACORN-O'],
                'stretchedSociety' : ['ACORN-P', 'ACORN-Q'],
                'lowIncomeLiving' : ['ACORN-U']}

In [10]:
luxury_life = ['ACORN-A', 'ACORN-B', 'ACORN-C']
established_affluence = ['ACORN-D', 'ACORN-E']
thriving_neighbour = ['ACORN-F', 'ACORN-G', 'ACORN-H', 'ACORN-I', 'ACORN-J']
steadfast_communities = ['ACORN-K','ACORN-L', 'ACORN-M', 'ACORN-N', 'ACORN-O']
stretched_society = ['ACORN-P', 'ACORN-Q']
low_income_living = ['ACORN-U']


In [11]:
np.random.seed(10)
for key in acorn_dict.keys():
    print(info_dataset[info_dataset.Acorn.isin(acorn_dict[key])].Acorn.value_counts())

NameError: name 'info_dataset' is not defined

In [12]:
def get_acorn_houses_list(acorn_dict,size=5,seed=10):
    info_dataset = pd.read_csv(f'informations_households.csv')
    np.random.seed(seed)
    house_list = []
    for key in acorn_dict.keys():
        sample = info_dataset[info_dataset.Acorn.isin(acorn_dict[key])].sample(size)
        for i in range(len(sample)):
            house_list.append(key + '_' +  sample.iloc[i,-1] + '_' + sample.iloc[i,0])
    
    return house_list

house_list = get_acorn_houses_list(acorn_dict,size=5,seed=10)

In [13]:
for house in house_list:
    block = house.split('_')[1] + '_' + house.split('_')[2]
    house_selected = house.split('_')[-1]
    print(block, house_selected)

block_0 MAC003737
block_5 MAC004664
block_3 MAC004417
block_6 MAC003349
block_4 MAC003762
block_37 MAC000861
block_34 MAC002265
block_21 MAC004531
block_23 MAC004137
block_22 MAC004736
block_70 MAC000360
block_50 MAC002586
block_66 MAC003655
block_69 MAC003784
block_71 MAC002468
block_88 MAC001727
block_85 MAC003618
block_75 MAC000343
block_85 MAC002272
block_89 MAC001380
block_107 MAC001673
block_107 MAC001672
block_94 MAC000870
block_109 MAC000529
block_107 MAC002426
block_111 MAC003860
block_111 MAC004828
block_110 MAC004069
block_110 MAC003780
block_110 MAC003977


In [22]:
def get_household(house):
    block = house.split('_')[1] + '_' + house.split('_')[2]
    house_selected = house.split('_')[-1]
    holidays = pd.read_csv('uk_bank_holidays.csv')
    holidays['Bank holidays'] = pd.to_datetime(holidays['Bank holidays'])
    holidays = holidays.loc[(holidays['Bank holidays'].dt.year >= 2013)]
    df = pd.read_csv(f'halfhourly_dataset/halfhourly_dataset/{block}.csv')
    df = df.rename(columns={'LCLid':'house_hold','tstp':'time','energy(kWh/hh)':'Energy_kwh'})
    df.iloc[df[df.Energy_kwh == 'Null'].Energy_kwh.index,2] = '0'
    df.Energy_kwh = pd.to_numeric(df.Energy_kwh)
    df.time = pd.to_datetime(df.time)
    df = df.loc[((df['time'].dt.year >= 2013) & (df.time.dt.month.isin([1,2,3,4,5,6,7,8,9,10,11,12]))) | (df['time'].dt.year == 2014)]
    df['holiday'] = (df.time.dt.day_of_week >= 10) | (df['time'].isin(holidays['Bank holidays']))
    df['holiday'] = df['holiday'].astype(int)
    df = df[df.house_hold == house_selected]
    
    return df, house

In [23]:
df = get_household(house_list[0])
df

(       house_hold                time  Energy_kwh  holiday
 759511  MAC003737 2013-01-01 00:00:00       0.088        1
 759512  MAC003737 2013-01-01 00:30:00       0.109        0
 759513  MAC003737 2013-01-01 01:00:00       0.075        0
 759514  MAC003737 2013-01-01 01:30:00       0.076        0
 759515  MAC003737 2013-01-01 02:00:00       0.108        0
 ...           ...                 ...         ...      ...
 779810  MAC003737 2014-02-27 22:00:00       0.063        0
 779811  MAC003737 2014-02-27 22:30:00       0.046        0
 779812  MAC003737 2014-02-27 23:00:00       0.086        0
 779813  MAC003737 2014-02-27 23:30:00       0.047        0
 779814  MAC003737 2014-02-28 00:00:00       0.083        0
 
 [20304 rows x 4 columns],
 'luxuryLife_block_0_MAC003737')

In [16]:
pd.read_csv(os.getcwd() + '\\' + 'weather_hourly_darksky.csv')

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2,3.70,98,9.46,2011-11-11 02:00:00,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
3,3.12,99,9.23,2011-11-11 03:00:00,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
4,1.85,111,9.26,2011-11-11 04:00:00,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy
...,...,...,...,...,...,...,...,...,...,...,...,...
21160,12.68,247,7.39,2014-02-15 19:00:00,3.06,997.07,3.91,6.08,rain,partly-cloudy-night,0.74,Partly Cloudy
21161,13.78,249,6.56,2014-02-15 20:00:00,2.76,998.15,3.03,5.61,rain,clear-night,0.77,Clear
21162,14.31,248,6.47,2014-02-15 21:00:00,2.71,999.28,3.06,5.25,rain,clear-night,0.77,Clear
21163,14.31,248,5.96,2014-02-15 22:00:00,2.69,1000.33,2.68,4.69,rain,clear-night,0.80,Clear


In [20]:
holidays = pd.read_csv('uk_bank_holidays.csv')
holidays['Bank holidays'] = pd.to_datetime(holidays['Bank holidays'])
holidays = holidays.loc[(holidays['Bank holidays'].dt.year >= 2013)]
df['holiday'] = (df.time.dt.day_of_week >= 10) | (df['time'].isin(holidays['Bank holidays']))
df['holiday'] = df['holiday'].astype(int)
df

AttributeError: 'tuple' object has no attribute 'time'

In [ ]:
['establishedAffluence_block_21_MAC004531.csv', 'establishedAffluence_block_22_MAC004736.csv', 'establishedAffluence_block_23_MAC004137.csv', 
 'establishedAffluence_block_34_MAC002265.csv', 'establishedAffluence_block_37_MAC000861.csv', 'lowIncomeLiving_block_110_MAC003780.csv', 
 'lowIncomeLiving_block_110_MAC003977.csv', 'lowIncomeLiving_block_110_MAC004069.csv', 'lowIncomeLiving_block_111_MAC003860.csv', 
 'lowIncomeLiving_block_111_MAC004828.csv', 'luxuryLife_block_0_MAC003737.csv', 'luxuryLife_block_3_MAC004417.csv', 
 'luxuryLife_block_4_MAC003762.csv', 'luxuryLife_block_5_MAC004664.csv', 'luxuryLife_block_6_MAC003349.csv', 
 'steadfastCommunities_block_75_MAC000343.csv', 'steadfastCommunities_block_85_MAC002272.csv', 'steadfastCommunities_block_85_MAC003618.csv', 
 'steadfastCommunities_block_88_MAC001727.csv', 'steadfastCommunities_block_89_MAC001380.csv', 'stretchedSociety_block_107_MAC001672.csv', 
 'stretchedSociety_block_107_MAC001673.csv', 'stretchedSociety_block_107_MAC002426.csv', 'stretchedSociety_block_109_MAC000529.csv', 
 'stretchedSociety_block_94_MAC000870.csv', 'thrivingNeighbour_block_50_MAC002586.csv', 'thrivingNeighbour_block_66_MAC003655.csv', 
 'thrivingNeighbour_block_69_MAC003784.csv', 'thrivingNeighbour_block_70_MAC000360.csv', 'thrivingNeighbour_block_71_MAC002468.csv']